In [2]:
import pandas as pd
import numpy as np
import glob

### Carregando a base de dados de consumo de energia elétrica

In [3]:
def load_and_concat_files_on_dir_energy(path: str, sep : str) -> pd.DataFrame:
    """
    Load all csv files on a directory and concat them in a single dataframe

    Parameters
    ----------
    path : str
        Path to the directory
    sep : str
        Separator of the files

    Returns
    -------
    pd.DataFrame
        Dataframe with all the files concatenated
    """
    all_files : list = glob.glob(path + f"/*.csv")
    li : list = []
    for filename in all_files:
        print(filename)
        df : pd.DataFrame = pd.read_csv(filename, index_col=None, header=0,sep=sep)
        li.append(df)
    df_carga_full : pd.DataFrame = pd.concat(li, axis=0, ignore_index=True)
    print(f"{len(li)} files loaded")
    return df_carga_full

path : str = './data/curva_carga'
df_carga_full : pd.DataFrame = load_and_concat_files_on_dir_energy(path, ';')
df_carga_full.shape


./data/curva_carga\CURVA_CARGA_2012.csv
./data/curva_carga\CURVA_CARGA_2013.csv
./data/curva_carga\CURVA_CARGA_2014.csv
./data/curva_carga\CURVA_CARGA_2015.csv
./data/curva_carga\CURVA_CARGA_2016.csv
./data/curva_carga\CURVA_CARGA_2017.csv
./data/curva_carga\CURVA_CARGA_2018.csv
./data/curva_carga\CURVA_CARGA_2019.csv
./data/curva_carga\CURVA_CARGA_2020.csv
./data/curva_carga\CURVA_CARGA_2021.csv
./data/curva_carga\CURVA_CARGA_2022.csv
11 files loaded


(357136, 4)

In [4]:
def create_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create time features from the timestamp
    Parameters
    ----------
    df : pd.DataFrame
        Dataframe with the timestamp

    Returns
    -------
    pd.DataFrame
        Dataframe with the time features
    """

    df_carga_full = df

    df_carga_full['din_instante'] = pd.to_datetime(df_carga_full['din_instante'])

    df_carga_full['date'] = df_carga_full['din_instante'].dt.date
    df_carga_full['year'] = df_carga_full['din_instante'].dt.year
    df_carga_full['month'] = df_carga_full['din_instante'].dt.month
    df_carga_full['day'] = df_carga_full['din_instante'].dt.day
    df_carga_full['hour'] = df_carga_full['din_instante'].dt.hour

    print(df_carga_full.info())

    return df_carga_full


df_carga_full : pd.DataFrame = create_time_features(df_carga_full)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357136 entries, 0 to 357135
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   id_subsistema            357136 non-null  object        
 1   nom_subsistema           357136 non-null  object        
 2   din_instante             357136 non-null  datetime64[ns]
 3   val_cargaenergiahomwmed  356397 non-null  float64       
 4   date                     357136 non-null  object        
 5   year                     357136 non-null  int64         
 6   month                    357136 non-null  int64         
 7   day                      357136 non-null  int64         
 8   hour                     357136 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 24.5+ MB
None


In [5]:
## Filtrando apenas os dados do sudeste

df_carga_full_sudeste : pd.DataFrame = df_carga_full[df_carga_full['nom_subsistema'] == 'SUDESTE']
print(df_carga_full_sudeste.shape)
df_carga_full_sudeste.head()
df_carga_full_sudeste.to_csv('./data/checkpoints/df_carga_v1.csv', sep=';')


(89326, 9)


## Carregando os dados climáticos

Usar como referência esse mapa : https://mapas.inmet.gov.br/, para pegar algumas das estações metereológicas para juntar os dados, e depois fazer as médias

Estações escolhidas : A740, A714, A739, A768, F501, A506 , A713 , A602  

A740 : -23.23, -45.42  
A714 : -23.98, -48.89  
A739 : -22.41, -46.81  
A768 : -21.93, -50.49  
F501 : -19.98, -43.96  
A506 : -16.69, -43.84 (descartar essa)  
A713 : -23.43, -47.59  
A602 : -23.05, -43.60  
A771 : 	-23.72, -46.68  
A634 : 	-20.47, -40.40  
A755 : -23.52, -46.87 (Barueri, bem completa)  
A733 : -20.16, -50.59 (Jales)  

In [26]:
def ajust_corrupted_float_value(value : str) -> float:
    """
    Função para corrigir valores com ponto flutuante que não estão no formato correto.

    Parameters
    ----------
    value : str
        Valor a ser corrigido.

    Returns
    -------
    float
        Valor corrigido.
    """
    try:
        value : str = str(value)
        if ',' in value:
            value : str = value.replace(',', '.')
        return float(value)
    except ValueError:
        if value[0] == ',':
            value : str = '0' + value
            value : float = float(value)
        else:
            raise ValueError("Erro ao converter precipitacao_total : Situação não prevista")
            
        return value


In [ ]:
%%time

def load_and_concat_files_on_dir_wheater(path: str, sep : str) -> pd.DataFrame:
    """
    Load all csv files on a directory and concat them in a single dataframe

    Parameters
    ----------
    path : str
        Path to the directory
    sep : str
        Separator of the files

    Returns
    -------
    pd.DataFrame
        Dataframe with all the files concatenated
    """
    all_files : list = glob.glob(path + f"/*.csv")
    li : list = []
    # allowed_vars : list = ['A740','A714','A739','F501','A713','A602','A771','A634']
    allowed_vars : list = ['A740','A714','A739','F501','A713','A602','A634']

    new_df : pd.DataFrame = pd.DataFrame()
    
    for filename in all_files:
        # print(filename)
        if filename[19:23] in allowed_vars:

            # Carregando o dataframe
            df : pd.DataFrame = pd.read_csv(filename, index_col=None,sep=sep,skiprows=10)

            # Renomeando as colunas
            df.rename(
                columns={
                    'PRECIPITACAO TOTAL, HORARIO(mm)':'precipitacao_total',
                    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA(mB)':'pressao_atmosferica',
                    'RADIACAO GLOBAL(Kj/m²)':'radiacao_global',
                    'TEMPERATURA DO AR - BULBO SECO, HORARIA(°C)':'temperatura_ar',
                    'TEMPERATURA DO PONTO DE ORVALHO(°C)':'temperatura_ponto_orvalho',
                    'UMIDADE RELATIVA DO AR, HORARIA(%)':'umidade_relativa_ar',
                    'VENTO, DIRECAO HORARIA (gr)(° (gr))':'vento_direcao',
                    'VENTO, VELOCIDADE HORARIA(m/s)':'velocidade_vento',
                    'Data Medicao':'data_medicao',
                    'Hora Medicao':'hora_medicao',
                }, inplace=True)

            # Criando índices de data
            df['data_medicao'] = pd.to_datetime(df['data_medicao'])
            df['year'] = df['data_medicao'].dt.year

            # Filtrando apenas os anos maiores que 2012
            df : pd.DataFrame = df[df['year'] >= 2012]

            # Ajustantado a coluna de hora
            df['hora_medicao'] =df['hora_medicao'] / 100
            df['hora_medicao'] =df['hora_medicao'].astype(int)

            # Selecionando apenas as colunas que serão utilizadas
            df : pd.DataFrame = df[['data_medicao','hora_medicao','precipitacao_total','pressao_atmosferica','radiacao_global','temperatura_ar','temperatura_ponto_orvalho','umidade_relativa_ar','vento_direcao','velocidade_vento']]
            
            # Formatando a coluna como data-hora
            df['data-hora'] = pd.to_datetime(df['data_medicao'].astype(str) + df['hora_medicao'].astype(str), format='%Y-%m-%d%H')

            # Definindo o índice
            df : pd.DataFrame = df.set_index('data-hora')

            ## Ajustando os valores de ponto flutuante
            df['precipitacao_total'] = df.apply(lambda x: ajust_corrupted_float_value(x['precipitacao_total']), axis=1)
            df['velocidade_vento'] = df.apply(lambda x: ajust_corrupted_float_value(x['velocidade_vento']), axis=1)
            df['pressao_atmosferica'] = df.apply(lambda x: ajust_corrupted_float_value(x['pressao_atmosferica']), axis=1)
            df['radiacao_global'] = df.apply(lambda x: ajust_corrupted_float_value(x['radiacao_global']), axis=1)
            df['temperatura_ar'] = df.apply(lambda x: ajust_corrupted_float_value(x['temperatura_ar']), axis=1)
            df['temperatura_ponto_orvalho'] = df.apply(lambda x: ajust_corrupted_float_value(x['temperatura_ponto_orvalho']), axis=1)
            
            # Renomenando a coluna com o sufixo da estação
            df.columns = [f"{str(col)}_" + filename[19:23] for col in df.columns]

            # Salvando o dataframe
            df.to_csv(f'data/silver/df_{filename[19:23]}.csv')

            li.append(df)
        

    df_carga_full : pd.DataFrame = pd.concat(li, axis=1)

    print(f"{len(li)} files loaded")
    return df_carga_full


path : str = './data/clima'
df_wheater : pd.DataFrame = load_and_concat_files_on_dir_wheater(path, ';')
print(df_wheater.shape)

# df_wheater.tail(100000).to_csv('data/checkpoints/df_wheater_concat_raw.csv')
# df_wheater.to_csv('data/checkpoints/df_wheater_concat_raw.csv')




In [21]:
df_wheater.head()

,data_medicao_A602,hora_medicao_A602,precipitacao_total_A602,pressao_atmosferica_A602,radiacao_global_A602,temperatura_ar_A602,temperatura_ponto_orvalho_A602,umidade_relativa_ar_A602,vento_direcao_A602,velocidade_vento_A602,...,data_medicao_F501,hora_medicao_F501,precipitacao_total_F501,pressao_atmosferica_F501,radiacao_global_F501,temperatura_ar_F501,temperatura_ponto_orvalho_F501,umidade_relativa_ar_F501,vento_direcao_F501,velocidade_vento_F501
data-hora,,,,,,,,,,,,,,,,,,,,,
2012-01-01 00:00:00,2012-01-01,0,"2,4","1007,3","-2,854","22,9",NaN,NaN,102.0,"4,3",...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 01:00:00,2012-01-01,1,"1,6","1007,6","-2,053","22,5",NaN,NaN,98.0,"2,2",...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 02:00:00,2012-01-01,2,6,"1006,9","-1,742","22,5",NaN,NaN,105.0,"2,5",...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 03:00:00,2012-01-01,3,"1,6",1006,"-2,066","22,6",NaN,NaN,99.0,"1,8",...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-01-01 04:00:00,2012-01-01,4,",6","1005,1","-1,992","22,7",NaN,NaN,57.0,",5",...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
def find_row_by_index(df : pd.DataFrame, date : str,) -> pd.DataFrame:
    """
    Função para encontrar uma linha de um dataframe a partir de um index
    
    Parameters
    ----------
    df : pd.DataFrame
        Dataframe a ser pesquisado
    date : str
        Index a ser pesquisado
    Returns
    -------
    pd.DataFrame
        Dataframe com a linha pesquisada
    """
    return df.filter(items=[date], axis=0)

find_row_by_index(df_wheater, '2012-12-31 21:00:00') # Filtrando a data especifica

,data_medicao_A602,hora_medicao_A602,precipitacao_total_A602,pressao_atmosferica_A602,radiacao_global_A602,temperatura_ar_A602,temperatura_ponto_orvalho_A602,umidade_relativa_ar_A602,vento_direcao_A602,velocidade_vento_A602,...,data_medicao_F501,hora_medicao_F501,precipitacao_total_F501,pressao_atmosferica_F501,radiacao_global_F501,temperatura_ar_F501,temperatura_ponto_orvalho_F501,umidade_relativa_ar_F501,vento_direcao_F501,velocidade_vento_F501
data-hora,,,,,,,,,,,,,,,,,,,,,
2012-12-31 21:00:00,2012-12-31,21,0,"1008,9","578,803",30,"21,3",59.0,42.0,"2,7",...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


KeyError: 'precipitacao_total'

### Testes e experimentos

In [ ]:
df : pd.DataFrame = pd.read_csv('./data/clima/dados_F501_H_2010-01-01_2021-12-31.csv', index_col=None,sep=";",skiprows=10)
print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/clima/dados_F501_H_2010-01-01_2021-12-31.csv'